<a href="https://colab.research.google.com/github/rutavshah/CSC413_Project/blob/main/CSC413_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing in MNIST Dataset

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def dataloader(batch_size):
  dataroot="/content/drive/My Drive/celeba"
  transform=transforms.Compose([ transforms.Resize(64),transforms.CenterCrop(64),transforms.ToTensor(),transforms.Normalize((0.5),(0.5))])
  dataset=torchvision.datasets.MNIST(root=dataroot, train=True, transform=transform, download=True)
  data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
  return data_loader


# Utils

In [ ]:
from torchvision.utils import make_grid , save_image
import numpy as np
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def show_and_save(file_name,img):
    npimg = np.transpose(img.numpy(),(1,2,0))
    # f = "./%s.png" % file_name
    fig = plt.figure(dpi=200)
    fig.suptitle(file_name, fontsize=14, fontweight='bold')
    plt.imshow(npimg)
    # plt.imsave(f,npimg)
def plot_loss(loss_list):
    plt.figure(figsize=(10,5))
    plt.title("Loss During Training")
    plt.plot(loss_list,label="Loss")
    
    plt.xlabel("iterations")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

# Decoder, Encoder/Generator, Discriminator

In [ ]:
import torch
import torch.nn as nn

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class Encoder(nn.Module):
  def __init__(self):
    super(Encoder,self).__init__()
    self.conv1=nn.Conv2d(1,64,5,padding=2,stride=2)   #in_channels=3
    self.bn1=nn.BatchNorm2d(64,momentum=0.9)
    self.conv2=nn.Conv2d(64,128,5,padding=2,stride=2)
    self.bn2=nn.BatchNorm2d(128,momentum=0.9)
    self.conv3=nn.Conv2d(128,256,5,padding=2,stride=2)
    self.bn3=nn.BatchNorm2d(256,momentum=0.9)
    self.relu=nn.LeakyReLU(0.2)
    self.fc1=nn.Linear(256*8*8,2048)
    self.bn4=nn.BatchNorm1d(2048,momentum=0.9)
    self.fc_mean=nn.Linear(2048,128)
    self.fc_logvar=nn.Linear(2048,128)   #latent dim=128
  
  def forward(self,x):
    batch_size=x.size()[0]
    out=self.relu(self.bn1(self.conv1(x)))
    out=self.relu(self.bn2(self.conv2(out)))
    out=self.relu(self.bn3(self.conv3(out)))
    out=out.view(batch_size,-1)
    out=self.relu(self.bn4(self.fc1(out)))
    mean=self.fc_mean(out)
    logvar=self.fc_logvar(out)
    
    return mean,logvar
class Decoder(nn.Module):
  def __init__(self):
    super(Decoder,self).__init__()
    self.fc1=nn.Linear(128,8*8*256)
    self.bn1=nn.BatchNorm1d(8*8*256,momentum=0.9)
    self.relu=nn.LeakyReLU(0.2)
    self.deconv1=nn.ConvTranspose2d(256,256,6, stride=2, padding=2)
    self.bn2=nn.BatchNorm2d(256,momentum=0.9)
    self.deconv2=nn.ConvTranspose2d(256,128,6, stride=2, padding=2)
    self.bn3=nn.BatchNorm2d(128,momentum=0.9)
    self.deconv3=nn.ConvTranspose2d(128,32,6, stride=2, padding=2)
    self.bn4=nn.BatchNorm2d(32,momentum=0.9)
    self.deconv4=nn.ConvTranspose2d(32,1,5, stride=1, padding=2)
    self.tanh=nn.Tanh()

  def forward(self,x):
    batch_size=x.size()[0]
    x=self.relu(self.bn1(self.fc1(x)))
    x=x.view(-1,256,8,8)
    x=self.relu(self.bn2(self.deconv1(x)))
    x=self.relu(self.bn3(self.deconv2(x)))
    x=self.relu(self.bn4(self.deconv3(x)))
    x=self.tanh(self.deconv4(x))
    return x
    
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.conv1=nn.Conv2d(1,32,5,padding=2,stride=1)
    self.relu=nn.LeakyReLU(0.2)
    self.conv2=nn.Conv2d(32,128,5,padding=2,stride=2)
    self.bn1=nn.BatchNorm2d(128,momentum=0.9)
    self.conv3=nn.Conv2d(128,256,5,padding=2,stride=2)
    self.bn2=nn.BatchNorm2d(256,momentum=0.9)
    self.conv4=nn.Conv2d(256,256,5,padding=2,stride=2)
    self.bn3=nn.BatchNorm2d(256,momentum=0.9)
    self.fc1=nn.Linear(8*8*256,512)
    self.bn4=nn.BatchNorm1d(512,momentum=0.9)
    self.fc2=nn.Linear(512,1)
    self.sigmoid=nn.Sigmoid()

  def forward(self,x):
    batch_size=x.size()[0]
    x=self.relu(self.conv1(x))
    x=self.relu(self.bn1(self.conv2(x)))
    x=self.relu(self.bn2(self.conv3(x)))
    x=self.relu(self.bn3(self.conv4(x)))
    x=x.view(-1,256*8*8)
    x1=x;
    x=self.relu(self.bn4(self.fc1(x)))
    x=self.sigmoid(self.fc2(x))

    return x,x1


# VAE-GAN

In [ ]:
class VAE_GAN(nn.Module):
  def __init__(self):
    super(VAE_GAN,self).__init__()
    self.encoder=Encoder()
    self.decoder=Decoder()
    self.discriminator=Discriminator()
    self.encoder.apply(weights_init)
    self.decoder.apply(weights_init)
    self.discriminator.apply(weights_init)


  def forward(self,x):
    bs=x.size()[0]
    z_mean,z_logvar=self.encoder(x)
    std = z_logvar.mul(0.5).exp_()
        
    #sampling epsilon from normal distribution
    epsilon=torch.randn(bs,128).to(device)
    z=z_mean+std*epsilon
    x_tilda=self.decoder(z)
      
    return z_mean,z_logvar,x_tilda

# GAN

In [ ]:
class GAN(nn.Module):
  def __init__(self):
    super(GAN,self).__init__()
    self.decoder=Decoder()
    self.discriminator=Discriminator()
    self.decoder.apply(weights_init)
    self.discriminator.apply(weights_init)

  def forward(self,z):

    x_tilda=self.decoder(z)
    x, x1 = self.discriminator(x_tilda)
    
    return x_tilda, x, x1

# VAE

In [ ]:
class VAE(nn.Module):
  def __init__(self):
    super(VAE,self).__init__()
    self.encoder=Encoder()
    self.decoder=Decoder()
    self.encoder.apply(weights_init)
    self.decoder.apply(weights_init)


  def forward(self,x):
    bs=x.size()[0]
    z_mean,z_logvar=self.encoder(x)
    std = z_logvar.mul(0.5).exp_()
        
    #sampling epsilon from normal distribution
    epsilon=torch.randn(bs,128).to(device)
    z=z_mean+std*epsilon
    x_tilda=self.decoder(z)
      
    return z_mean,z_logvar,x_tilda

# Training: VAE-GAN

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.autograd.set_detect_anomaly(True)

data_loader=dataloader(64)
gen=VAE_GAN().to(device)
discrim=Discriminator().to(device)
real_batch = next(iter(data_loader))
show_and_save("training" ,make_grid((real_batch[0]*0.5+0.5).cpu(),8))

epochs=3
lr=3e-4
alpha=0.1
gamma=15

# Loss function and Gradient descent algorithms
criterion = nn.BCELoss().to(device)
optim_E = torch.optim.RMSprop(gen.encoder.parameters(), lr=lr)
optim_D = torch.optim.RMSprop(gen.decoder.parameters(), lr=lr)
optim_Dis = torch.optim.RMSprop(discrim.parameters(), lr=lr*alpha)

z_fixed = torch.randn((64,128)).to(device)
x_fixed = real_batch[0].to(device)

for epoch in range(epochs):
  prior_loss_list,gan_loss_list,recon_loss_list=[],[],[]
  dis_real_list,dis_fake_list,dis_prior_list=[],[],[]
  for i, (data,_) in enumerate(data_loader, 0):
    bs=data.size()[0]
    
    ones_label=torch.ones(bs,1).to(device)
    zeros_label=torch.zeros(bs,1).to(device)
    zeros_label1=torch.zeros(64,1).to(device)
    datav = data.to(device)

    mean, logvar, rec_enc = gen(datav)

    # Random z that is input for the GAN generator or the VAE-GAN 
    z_p = torch.randn(64,128).to(device)
    x_p_tilda = gen.decoder(z_p)
 
    output = discrim(datav)[0]
    errD_real = criterion(output, ones_label)
    dis_real_list.append(errD_real.item())

    output = discrim(rec_enc)[0]
    errD_rec_enc = criterion(output, zeros_label)
    dis_fake_list.append(errD_rec_enc.item())

    output = discrim(x_p_tilda)[0]
    errD_rec_noise = criterion(output, zeros_label1)
    dis_prior_list.append(errD_rec_noise.item())

    gan_loss = errD_real + errD_rec_enc + errD_rec_noise
    gan_loss_list.append(gan_loss.item())

    # Gradient descent update for Discrimator
    optim_Dis.zero_grad()
    gan_loss.backward(retain_graph=True)
    optim_Dis.step()


    output = discrim(datav)[0]
    errD_real = criterion(output, ones_label)

    output = discrim(rec_enc)[0]
    errD_rec_enc = criterion(output, zeros_label)

    output = discrim(x_p_tilda)[0]
    errD_rec_noise = criterion(output, zeros_label1)

    # GAN Loss
    gan_loss = errD_real + errD_rec_enc + errD_rec_noise
    
    
    x_l_tilda = discrim(rec_enc)[1]
    x_l = discrim(datav)[1]
    rec_loss = ((x_l_tilda - x_l) ** 2).mean()
    err_dec = gamma * rec_loss - gan_loss 
    recon_loss_list.append(rec_loss.item())

    # Gradient descent update for Decoder/Generator
    optim_D.zero_grad()
    err_dec.backward(retain_graph=True)
    optim_D.step()
    
    mean, logvar, rec_enc = gen(datav)
    x_l_tilda = discrim(rec_enc)[1]
    x_l = discrim(datav)[1]
    rec_loss = ((x_l_tilda - x_l) ** 2).mean()
    prior_loss = 1 + logvar - mean.pow(2) - logvar.exp()
    prior_loss = (-0.5 * torch.sum(prior_loss))/torch.numel(mean.data)
    prior_loss_list.append(prior_loss.item())
    err_enc = prior_loss + 5*rec_loss

    # Gradient descent update for Encoder
    optim_E.zero_grad()
    err_enc.backward(retain_graph=True)
    optim_E.step()
    
    if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_gan: %.4f\tLoss_prior: %.4f\tRec_loss: %.4f\tdis_real_loss: %0.4f\tdis_fake_loss: %.4f\tdis_prior_loss: %.4f'
                  % (epoch,epochs, i, len(data_loader),
                     gan_loss.item(), prior_loss.item(),rec_loss.item(),errD_real.item(),errD_rec_enc.item(),errD_rec_noise.item()))

  b=gen(x_fixed)[2]
  b=b.detach()
  c=gen.decoder(z_fixed)
  c=c.detach()
  show_and_save('MNISTrec_noise_epoch_%d.png' % epoch ,make_grid((c*0.5+0.5).cpu(),8))
  show_and_save('MNISTrec_epoch_%d.png' % epoch ,make_grid((b*0.5+0.5).cpu(),8))

plot_loss(prior_loss_list)
plot_loss(recon_loss_list)
plot_loss(gan_loss_list)
torch.save(gen.state_dict(), '/content/gen' )
torch.save(discrim.state_dict(),'/content/discrim' )

# Training: GAN

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.autograd.set_detect_anomaly(True)

data_loader=dataloader(64)

gan = GAN().to(device)
real_batch = next(iter(data_loader))
# show_and_save("training" ,make_grid((real_batch[0]*0.5+0.5).cpu(),8))

epochs=1
lr=0.04
alpha=0.1
gamma=15
iters_d = 2

# Loss function
criterion = nn.BCELoss().to(device)

# Gradient Descent algorithms
GAN_optim_Gen = torch.optim.RMSprop(gan.decoder.parameters(), lr=lr)
GAN_optim_Dis = torch.optim.RMSprop(gan.discriminator.parameters(), lr=lr*alpha)

for epoch in range(epochs):
 
  for i, (data,_) in enumerate(data_loader, 0):
    bs=data.size()[0]
    
    ones_label=torch.ones(bs,1).to(device)
    zeros_label=torch.zeros(bs,1).to(device)
    zeros_label1=torch.zeros(64,1).to(device)
    datav = data.to(device)

    # Training step for the Discriminator
    for j in range(iters_d):
      z = torch.randn(64,128).to(device)
      x_tilde, x, x1 = gan(z)
      x_real, x2 = gan.discriminator(datav)

      # Loss for Discriminator
      dis_loss =  criterion(x_real, ones_label) + criterion(x, zeros_label1)

      # Gradient Descent update for the Discriminator
      GAN_optim_Dis.zero_grad()
      dis_loss.backward(retain_graph=True)
      GAN_optim_Dis.step()

    # Training step for the Generator

    z = torch.randn(64,128).to(device)
    x_tilde, x, x1 = gan(z)

    # Loss for Generator
    gen_loss = criterion(x, zeros_label1)

    # Gradient Descent update for the Generator
    GAN_optim_Gen.zero_grad()
    gen_loss.backward(retain_graph=True)
    GAN_optim_Gen.step()


    if i % 50 == 0:
      print("[%d/%d][%d/%d]" % (epoch,epochs, i, len(data_loader)))

torch.save(gan.state_dict(), '/content/GAN_model' )


[0/1][0/938]
[0/1][50/938]
[0/1][100/938]


# Training: VAE

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.autograd.set_detect_anomaly(True)

data_loader=dataloader(64)

vae = VAE().to(device)
real_batch = next(iter(data_loader))
# show_and_save("training" ,make_grid((real_batch[0]*0.5+0.5).cpu(),8))

epochs=1
lr=3e-4
alpha=0.1
gamma=15

VAE_optim = torch.optim.RMSprop(vae.parameters(), lr=lr)

for epoch in range(epochs):
 
  for i, (data,_) in enumerate(data_loader, 0):
    bs=data.size()[0]

    datav = data.to(device)

    z_mean,z_logvar,x_tilda = vae(datav)

    # Calculating the loss (Element wise and KLD)
    rec_loss = ((x_tilda - datav) ** 2).mean()
    prior_loss = 1 + z_logvar - z_mean.pow(2) - z_logvar.exp()
    prior_loss2 = (-0.5 * torch.sum(prior_loss))/torch.numel(z_mean.data)
    err_enc = prior_loss2 + rec_loss

    # Gradient descent update
    VAE_optim.zero_grad()
    err_enc.backward(retain_graph=True)
    VAE_optim.step()

    if i % 50 == 0:
      print("[%d/%d][%d/%d]" % (epoch,epochs, i, len(data_loader)))

torch.save(vae.state_dict(), '/content/VAE_model' )

# Example: VAE-GAN Model 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gen2 = VAE_GAN()
gen2.load_state_dict(torch.load('/content/modela', map_location=torch.device(device)))

data_loader=dataloader(64)
real_batch = next(iter(data_loader))
z_fixed = torch.randn((64,128))
x_fixed = real_batch[0].to(device)

z_mean,z_logvar,x_tilda = gen2(x_fixed)
x_tilda=x_tilda.detach()
print(x_tilda)

show_and_save('MNISTrec_epoch.png'  ,make_grid((x_tilda*0.5+0.5).cpu(),8))
# show_and_save('MNISTrec_epoch.png'  ,make_grid((x_fixed*0.5+0.5).cpu(),8))

# Example: GAN Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gan2 = GAN()
real_batch = next(iter(data_loader))
gan2.load_state_dict(torch.load('/content/GAN_model', map_location=torch.device(device)))

data_loader=dataloader(64)
z_fixed = torch.randn((64,128))
x_fixed = real_batch[0].to(device)


real_batch = next(iter(data_loader))
x_tilda, x, x1 = gan2(z_fixed)
x_tilda=x_tilda.detach()
print(x_tilda)

show_and_save('MNISTrec_epoch.png'  ,make_grid((x_tilda*0.5+0.5).cpu(),8))
show_and_save('MNISTrec_epoch.png'  ,make_grid((x_fixed*0.5+0.5).cpu(),8))

# Example: VAE Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vae2 = VAE().to(device)
real_batch = next(iter(data_loader))
vae2.load_state_dict(torch.load('/content/VAE_model', map_location=torch.device(device)))

data_loader=dataloader(64)
z_fixed = torch.randn((64,128))
x_fixed = real_batch[0].to(device)


real_batch = next(iter(data_loader))
z_mean,z_logvar,x_tilda = vae2(x_fixed)
x_tilda=x_tilda.detach()
print(x_tilda)

show_and_save('MNISTrec_epoch.png'  ,make_grid((x_tilda*0.5+0.5).cpu(),8))
# show_and_save('MNISTrec_epoch.png'  ,make_grid((x_fixed*0.5+0.5).cpu(),8))